In [1]:
import sys
sys.path.append('./')
import numpy as np
import pandas as pd
import neural_tape_controller
#Положительные числа - положительные награды.
import tasks 
import optimize
import pickle
root_dir = './'

In [3]:
mem_size = 100
heads_mem = 2
input_size=36
output_size=5

In [5]:
layers_desc = [#{'type':'gru','out':input_size+4,'cells':5,'activation':'lrelu'},
                #{'type':'conv','filter_size':[3,1,3,3],'stride':1,'padding':0,'activation':'lrelu'},
                #{'type':'max_pool','pool_size':2,'stride':2,'activation':'lrelu'},
                #{'type':'conv','filter_size':[1,1,3,3],'stride':1,'padding':0,'activation':'lrelu'},
                #{'type':'max_pool','pool_size':2,'stride':2,'activation':'lrelu'},
                #{'type':'conv','filter_size':[1,1,3,3],'stride':1,'padding':0,'activation':'lrelu'},
                #{'type':'max_pool','pool_size':2,'stride':2,'activation':'lrelu'},
                #{'type':'conv','filter_size':[1,1,3,3],'stride':1,'padding':0,'activation':'lrelu'},
                #{'type':'max_pool','pool_size':2,'stride':2,'activation':'lrelu'},
                #{'type':'flatten','out':88,'activation':'linear'},
                #{'type':'ff','out':60,'activation':'lrelu'},
                {'type':'ff','out':36,'activation':'lrelu'},
                {'type':'modulable','out':36,'name':'mod0','activation':'linear'},
                {'type':'ff','out':36,'activation':'lrelu'},
                {'type':'modulator_inertial','out':36,'name':'mod0','activation':'linear'},
                {'type':'ff','out':60,'activation':'lrelu'},
                {'type':'modulable','out':60,'name':'mod1','activation':'linear'},
                {'type':'turing_read','name':'memory_tape','heads':heads_mem,'cells':mem_size,'activation':'lrelu'},
                {'type':'ff','out':60,'activation':'lrelu'},
                {'type':'turing_move','name':'memory_tape','heads':heads_mem,'cells':mem_size},
                {'type':'turing_write','name':'memory_tape','heads':heads_mem,'cells':mem_size},           
                {'type':'ff','out':60,'activation':'lrelu'},
                {'type':'modulator_inertial','out':60,'name':'mod1','activation':'linear'},
                {'type':'ff','out':160,'activation':'lrelu'},                           
                {'type':'modulable','out':160,'name':'mod2','activation':'linear'},
                {'type':'ff','out':160,'activation':'lrelu'},
                {'type':'modulator_inertial','out':160,'name':'mod2','activation':'linear'},
                {'type':'ff','out':160,'activation':'lrelu'},
                {'type':'modulable','out':160,'name':'mod3','activation':'linear'},
                {'type':'ff','out':160,'activation':'lrelu'},
                {'type':'modulator_inertial','out':160,'name':'mod3','activation':'linear'},
                {'type':'ff','out':160,'activation':'lrelu'},
                {'type':'modulable','out':160,'name':'mod4','activation':'linear'},
                {'type':'ff','out':160,'activation':'lrelu'},
                {'type':'modulator_inertial','out':160,'name':'mod4','activation':'linear'},
                {'type':'ff','out':160,'activation':'lrelu'},
                {'type':'modulable','out':160,'name':'mod5','activation':'linear'},
                {'type':'ff','out':160,'activation':'lrelu'},
                {'type':'modulator_inertial','out':160,'name':'mod5','activation':'linear'},
                {'type':'ff','out':output_size,'activation':'lrelu'}]

In [6]:
nt = neural_tape_controller.nt_controller(tacts=1,input_size=input_size,output_size=output_size,layers_desc=layers_desc)
genom = nt.nn.disassemble_genom()

In [5]:
try:
    with open(root_dir+f'./genoms/best_genoms_aa_gun2.pkl', 'rb') as f:
        genom = pickle.load(f)[-1]
    print('loaded successfully')
except Exception:
    pass

In [7]:
%%time
q=tasks.custom_test(genom, verbose=True, draw=False)
print('q=',q)

aa_gun_2: [0 0 0 0 0 0]  full=0.0
q= -9.98377799987793e-14
CPU times: user 13.6 s, sys: 286 µs, total: 13.6 s
Wall time: 13.6 s


In [ ]:
print(pd.Timestamp.now())

In [ ]:
%%time
n_jobs = 12
from multiprocessing import Pool
pool = Pool(processes=n_jobs)
y_lst = pool.map(tasks.custom_test, [x for x in [genom]*n_jobs])
pool.close()
pool.join()

In [8]:
print(1)
opt = optimize.optimizer(tasks.custom_test, genom_size=len(genom),parallel_cores=12,init_file='genoms/genom_gun.pkl',history_file='history/history_gun.pkl')

1
history loaded successfully


In [6]:
try:
    with open(root_dir+f'./genoms/best_genoms_aa_gun2.pkl', 'rb') as f:
        opt.best_genoms = pickle.load(f)
    print('loaded successfully')
except Exception:
    pass

loaded successfully


In [9]:
opt.function(opt.best_genoms[-1], verbose=True)
for i in range(1000):
    print('opt#',i)
    opt.optimize()
    with open(root_dir+f'./genoms/best_genoms_aa_gun2.pkl', 'wb') as f:
        pickle.dump(opt.best_genoms,f,protocol=pickle.HIGHEST_PROTOCOL)
        print('WRITTEN')

aa_gun_2: [0 0 0 0 0 0]  full=0.0
opt# 0
scores for optimizers augmented [-0.045      -0.0225     -0.025      -0.01475614  0.07053642 -0.042
 -0.075      -0.06647769 -0.068     ]
chosen gradient_long_adaptive_inertial previous_result: 0.0 per tacts: 136
score_new -3.3311452085649658e-12 score_prev -3.3311452085649654e-12 gained -4.0389678347315804e-28
undo


KeyboardInterrupt: 

In [ ]:
#draw
with open(root_dir+f'./genoms/best_genoms_aa_gun2.pkl', 'rb') as f:
    genom = pickle.load(f)
    genom = genom[-1]
print('loaded successfully')
q=tasks.custom_test(genom, verbose=True, draw=True)
print('q=',q)

In [ ]:
#save gif
tasks.video[0].save(
    root_dir+'./out_videos/best_genoms_aa_gun.gif',
    save_all=True,
    append_images=tasks.video[1:], 
    optimize=True,
    duration=200,
    loop=0
)

In [8]:
#draw video
import cv2
writer = cv2.VideoWriter("outputvideo.mp4",cv2.VideoWriter_fourcc(*"MP4V"),30,(512,512))
for i in range(0,len(tasks.video),1):
    resize = cv2.resize(np.array(tasks.video[i]), (512, 512))
    frame = resize*0
    frame[:,:,0],frame[:,:,1],frame[:,:,2] = resize[:,:,2],resize[:,:,1], resize[:,:,0]
    writer.write(frame)
writer.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
